<a href="https://colab.research.google.com/github/Duncanswilson/gpt4-code-generation/blob/main/cifar_tpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install required libraries
!pip install -q tensorflow numpy

# Step 2: Connect to a TPU runtime
%tensorflow_version 2.x
import tensorflow as tf
import os
try:
  device_name = os.environ['COLAB_TPU_ADDR']
  TPU_ADDRESS = 'grpc://' + device_name
  print('Found TPU at: {}'.format(TPU_ADDRESS))
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_ADDRESS)
  tf.config.experimental_connect_to_cluster(resolver)
  tf.tpu.experimental.initialize_tpu_system(resolver)
  strategy = tf.distribute.experimental.TPUStrategy(resolver)
except KeyError:
 print('TPU not found')

# Step 3: Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Step 4: Preprocess the data
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = tf.keras.utils.to_categorical(y_train), tf.keras.utils.to_categorical(y_test)

# Step 5: Build the DNN model
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

with strategy.scope():
    model = create_model()

# Step 6: Train the model on the TPU
model.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found TPU at: grpc://10.3.127.186:8470


170498071/170498071 [==============================] - 6s 0us/step
Epoch 1/10
380/391 [============================>.] - ETA: 0s - loss: 1.6787 - accuracy: 0.3815